## Data ETL Notebook

**Layer**: Silver

**Domain**: Risk-free

**Action**: Transform RBNZ Yields and Series Data - Change group field (years) to numeric, and yields as decimals e.g. #.#% becomes 0.0##

In [0]:
%sql
-- Read current records from Silver cleaned and merge Series data (overwrite current version)
CREATE OR REPLACE TABLE workspace.riskfree_silver.003_rbnz_yields_transformed AS
SELECT
    -- keep source columns
    date,
    series_id,
    yield_percent,
    source_file_name,
    ingestion_timestamp,
    group,
    series,
    unit,

    -- numeric term: extract the first number and
    --             convert months to years if present
    CASE
        WHEN lower(series) LIKE '%month%' THEN      -- e.g. '18 months'
             regexp_extract(series, '([0-9]+\\.?[0-9]*)', 1)  / 12
        ELSE                                       -- e.g. '1 year', '1.5 year'
             regexp_extract(series, '([0-9]+\\.?[0-9]*)', 1)
    END::double   AS term_years,

    -- 1.50  → 0.015
    yield_percent / 100.0  AS yield_decimal
FROM workspace.riskfree_silver.002_rbnz_yields_enriched
;

-- Confirmation message
SELECT '✅ Silver table created: workspace.riskfree_silver.003_rbnz_yields_transformed' AS message;